In [1]:
import os,sys
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import matplotlib.font_manager as fm
fm.fontManager.addfont('/home/qiuyang/workplace/world_map_drawing/Geographical heterogeneity/Geomap/bokeh-app/data/font/times.ttf')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi']=300
plt.rcParams['font.family']='Times New Roman' #please remove this line if font is not installed
plt.rcParams['font.sans-serif']='Times New Roman' # please remove this line if Arial font is not installed
plt.rcParams['pdf.fonttype']=42
from PyComplexHeatmap import *
sys.path.append(os.path.expanduser("~/Projects/Github/PyComplexHeatmap/"))
import PyComplexHeatmap as pch
print(pch.__version__)
from matplotlib.colors import LinearSegmentedColormap

df_corr = pd.read_csv("/home/qiuyang/workplace/world_map_drawing/Geographical heterogeneity/draw_data/hotmap/data/kycg_modules_correlations.csv",sep='\t',index_col=0)
df_ann = pd.read_csv("/home/qiuyang/workplace/world_map_drawing/Geographical heterogeneity/draw_data/hotmap/data/kycg_modules_annotations.csv",sep='\t')
betas = pd.read_csv("/home/qiuyang/workplace/world_map_drawing/Geographical heterogeneity/draw_data/hotmap/data/kycg_modules_betas.csv",sep='\t')
cpg_std=betas.std().to_dict()
cpg_mean=betas.mean().to_dict()

df_ann.set_index('CpG',inplace=True)
df_ann.Module=df_ann.Module.astype(str)

df_ann=df_ann.loc[df_ann.Module.isin(['4','1','3','9','2','39'])]
keep_cpgs=df_ann.index.tolist()
df_corr=df_corr.loc[keep_cpgs,keep_cpgs]
df_ann['Std']=df_ann.index.to_series().map(cpg_std)
df_ann['Mean']=df_ann.index.to_series().map(cpg_mean)
data=df_corr.stack().reset_index()
data.columns=['X','Y','Correlation']
data['Module']=data.X.map(df_ann.Module.to_dict())
data['ChromHMM']=data.X.map(df_ann.ChromHMM.to_dict())
print(df_ann)
# keep_hm=['H3K4me1','H3K4me3','H3K27me1','H3K27me3','H3K27me3B']
# for hm in keep_hm:
#     df_ann[hm]=df_ann.HM.fillna('').apply(lambda x:1 if hm in x.split(';') else 0)
#     data[hm]=data.X.map(df_ann[hm].to_dict())

# print(df_ann.shape)
# df_ann.head()


1.7.4
           Module ChromHMM ChromHMM_bioc  \
CpG                                        
cg04735237      1     TxWk          TxWk   
cg00643814      1    Quies         Quies   
cg24865495      1    Quies         Quies   
cg25376651      1    Quies         Quies   
cg27485084      1    Quies         Quies   
...           ...      ...           ...   
cg00565688     39   TssBiv        EnhBiv   
cg24073122     39   EnhBiv        EnhBiv   
cg12475507     39   EnhBiv        EnhBiv   
cg07382920     39   EnhBiv        EnhBiv   
cg17496659     39   TssBiv        EnhBiv   

                                              HM  \
CpG                                                
cg04735237             H3K4me1;H3K79me2;H3K79me3   
cg00643814                        H3F3A;H4K20me1   
cg24865495                                   NaN   
cg25376651                      H2BK20ac;H3K4me1   
cg27485084                        H3F3A;H3K23me2   
...                                          ...   
cg005

In [7]:
df_ann=df_ann.loc[df_ann.Module.isin(['4','1','3'])]
keep_cpgs=df_ann.index.tolist()
df_corr=df_corr.loc[keep_cpgs,keep_cpgs]

data=df_corr.stack().reset_index()

data.columns=['X','Y','Correlation']

#Module 是区块
data['Module']=data.X.map(df_ann.Module.to_dict())
#ChromHMM 是区块中的类型区分
data['ChromHMM']=data.X.map(df_ann.ChromHMM.to_dict())


#添加上四个列以及对应的01值
keep_hm=['H3K4me1','H3K4me3','H3K27me1','H3K27me3B']
for hm in keep_hm:
    df_ann[hm]=df_ann.HM.fillna('').apply(lambda x:1 if hm in x.split(';') else 0)
    data[hm]=data.X.map(df_ann[hm].to_dict())

row_ha = pch.HeatmapAnnotation(
            Module=pch.anno_simple(df_ann.Module,cmap='Dark2',legend=True,height=5,
                              add_text=True,text_kws={'color':'black','fontsize':12}),
            axis=0,verbose=0,label_kws={'visible':False})

all_cmaps=matplotlib.pyplot.colormaps()
if 'binarize' not in all_cmaps:
    c = LinearSegmentedColormap.from_list('binarize', [(0, 'lightgray'), (1, 'black')])
    try:
        plt.register_cmap(cmap=c)
    except:
        matplotlib.colormaps.register(c, force=True)


print(data['Correlation'])
# col_ha = pch.HeatmapAnnotation(
#             #label=pch.anno_label(df_col.ColGroup, merge=True,rotation=45),
#             Module=pch.anno_simple(df_ann.Module,cmap='Dark2',legend=True,height=5,
#                               add_text=True,text_kws={'color':'black','fontsize':12}),
#             ChromHMM=pch.anno_simple(df_ann.ChromHMM,cmap='tab20'),
#             Mean=pch.anno_barplot(df_ann.Mean,cmap='jet',linewidth=0.1),
#             # H3K27me3=pch.anno_simple(df_ann.H3K27me3,cmap='binarize',legend=False),
#             # H3K27me3B=pch.anno_simple(df_ann.H3K27me3B,cmap='binarize',legend=False),
#             verbose=0,label_side='right',label_kws={'horizontalalignment':'left'})

# plt.figure(figsize=(10, 9))
# cm = pch.DotClustermapPlotter(
#             data=data, x='X',y='Y',value='Correlation',c='Correlation',s='Correlation',
#             hue='Module', cmap='jet',#cmap={'High':'Reds','Middle':'Purples','Low':'Greens'},
#             #colors={'High':'red','Middle':'purple','Low':'green'},
#             #marker={'4':'P','1':'*','3':'D'},
#             top_annotation=col_ha,right_annotation=row_ha,
#             col_split=df_ann.Module,row_split=df_ann.Module, col_split_gap=1,row_split_gap=1,
#             row_dendrogram=False,legend_anchor="ax_heatmap",legend_hpad=7,legend_vpad=5,
#             tree_kws=dict(row_cmap='Dark2',colors='red',linewidth=2),verbose=0,legend_gap=7,alpha=2,spines=True)
# plt.savefig("dotClustermap2.pdf", bbox_inches='tight')
# plt.show()

0        1.000000
1        0.559916
2        0.686845
3        0.561415
4        0.699960
           ...   
10604    0.423810
10605    0.389660
10606    0.626528
10607    0.453287
10608    1.000000
Name: Correlation, Length: 10609, dtype: float64
